# Import Torch functions and tokenizers

In [1]:
import torch
from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import dataset
from torch.utils.tensorboard import SummaryWriter

import regex as re
import os
import time
from tqdm import tqdm
import copy
import math

from model import TransformerModel
from utils import preProcessText, getTokenizer,try_gpu , word_piece_decoder, word_piece_encoder
from config import getConfig

import pickle

In [2]:
# model_config, app_config = getConfig(small = True)
model_config, app_config = getConfig()
print(model_config)
print(app_config)

bptt=model_config["bptt"]
device = try_gpu(0)

{'emsize': 300, 'd_hid': 1024, 'nlayers': 6, 'nhead': 6, 'dropout': 0.2, 'bptt': 64}
{'logs': 'tensorboard_logs', 'epochs': 25}


# Preprocessing Text

In [3]:
# t1[:5000]

In [4]:

file_path = 'data/preprocessed_morph.txt'
if not os.path.exists(file_path):
    print("Run morpheme_datagen notebook")
else:
    print(f"Reading file  : {file_path}")
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        text = preProcessText(text,tokenizer_type = 'morpheme')

Reading file  : data/preprocessed_morph.txt


In [5]:
text[:5000]

'बर्दिबास * नगरपालिका को * तेस्रो * नगर * परिषदबाट * पारित * आव २०७३ * । * ७४ * को * संशोधित * र * २०७४ * । * ७५ * को * प्रस्तावित * नीति * ? * कार्यक्रम * तथा * बजेट\nअार्थिक * वर्ष * २०७५७६ * काे * नदिजन्य * पदार्थ काे * उत्खनन् * गरी * बिक्रि * वितरण * तथा * अान्तरिक * निकासी * गर्ने * कार्य काे * बाेलपत्र * सम्बन्धी * सुचना\nसक्ष ार * सप्तरी * अभियानमा * सप्तरीबासी * सम्पूर्ण * सरोकारवालाहरु को * सहयोग * र * सहभागिता का ो * लागि * अनुराोध * छ * । * सामुदायिक * अध्ययन * केन्द्र हरूको * नविकरण * सम्बन्धमा * । * \nकाठमाडौं * ? * १२ * कातिक * । * राष्ट्रपति * विद्यादेवी * भण्डारी * मित्रराष्ट्र * कतारको * चार * दिवसीय * औपचारिक * भ्रमणमा * आज * त्यसतर्फ * प्रस्थान * गरेकी * छन् * । * राष्ट्रपति * विद्या देवी * भण्डारी * कतारका * अमिर * शेख * ह मा द * बीन * खालिदा * अल * थानीको * मैत्री पूर्ण * निमन्त्रणामा * चार * दिवसीय * औपचारिक\nकाठमाडौँ * ? * २६ * कात्तिक * । * सरकारले * सङ्घ * ? * प्रदेश * र * स्थानीय * तहमा * कर्मचारी * समायोजन * गर्नका * लागि * कर्मचारी * समायोजन * अध्यादेश २०७५

In [6]:
len(text.split('\n'))

123563

In [7]:
train_iter_first = text.split('\n')[:100_000]
test_iter = text.split('\n')[100_000:]

# Run Here

In [8]:
# import pickle


# tokenizer = get_tokenizer(None)

# vocab = build_vocab_from_iterator(
#     map(tokenizer, train_iter_first), specials=['<unk>'],max_tokens = 30000)
# vocab.set_default_index(vocab['<unk>'])


# # Save for first time
# with open('transformer_vocab_morpheme.pickle','wb') as f:
#     pickle.dump(vocab,f)

tokenizer,vocab = getTokenizer(tokenizer_type = 'morpheme')

# with open('tokenizers/transformer_vocab_morpheme.pickle','rb') as f:
#     vocab = pickle.load(f)

In [9]:
#dir(vocab)

In [10]:
#vocab.get_itos()

In [11]:
sorted((vocab.get_stoi()))[:20]

['*',
 '+',
 '++',
 '<unk>',
 '?',
 'ँ',
 'ँदा',
 'ँदै',
 'ँदैछ',
 'ँदैछन्',
 'ँदैन',
 'ँदैनन्',
 'ँदो',
 'ं',
 'ंदा',
 'ंदै',
 'ंदैछ',
 'ंदैन',
 'ः',
 'ः३०']

In [12]:
len(vocab)

30000

#  some utility functions

In [13]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long)
            for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
#     data = data.view(bsz, seq_len).t().contiguous()
    data = data.view(bsz,seq_len).t()
#     return data.to(device)
    return data


seq_length = bptt
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(seq_length, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

In [14]:
#Train and Test Split
train_data = data_process(train_iter_first)
test_data = data_process(test_iter)

In [15]:
len(train_data)

44038548

In [16]:
train_data[:40]

tensor([14037,     1,   146,     4,     1,   572,     1,   778,     1,     0,
            1,   985,     1,  1879,  1520,     1,     2,     1,  3696,     1,
            4,     1, 11933,     1,     5,     1,   813,     1,     2,     1,
         1527,     1,     4,     1,  4469,     1,   366,     1,     3,     1])

# Working with a dummy Sample

In [17]:
#Sample Data


text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
text = ['। * सामुदायिक * अध्ययन * केन्द्र हरूको *']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [18]:
sample_data

tensor([   2,    1, 1079,    1,  290,    1,  197,  329,    1])

In [19]:
sample_data = batchify(sample_data, 3)
print("Given word:", text[0])
sample_data

Given word: । * सामुदायिक * अध्ययन * केन्द्र हरूको *


tensor([[   2,    1,  197],
        [   1,  290,  329],
        [1079,    1,    1]])

# Model Definition

In [20]:
# batched_train_data = batchify(train_data, bptt).to(device)  # shape [seq_len, batch_size]
# batched_test_data = batchify(test_data, bptt).to(device)

In [21]:
def get_model(model_config, ntokens):
    emsize = model_config["emsize"]
    d_hid = model_config["d_hid"]
    nlayers = model_config["nlayers"]
    nhead = model_config["nhead"]
    dropout = model_config["dropout"]
    model = TransformerModel(ntokens, emsize,nhead, d_hid, nlayers, dropout)
    return model

In [22]:
ntokens = len(vocab)
model = get_model(model_config, ntokens).to(device)
torch.cuda.memory_allocated()

103103488

In [23]:
batched_train_data = batchify(train_data, bptt).to(device)  # shape [seq_len, batch_size]
batched_test_data = batchify(test_data, bptt).to(device)

In [24]:
seq_length = bptt

# Hyper-Parameter Tuning

In [25]:
criterion = nn.CrossEntropyLoss()
lr = 1  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
softmax = nn.Softmax(dim=2)
#softmax = nn.LogSoftmax(dim=2)

In [26]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [27]:
def train(model: nn.Module) -> None:
    global epoch
    global global_step
    model.train()  # turn on train mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(batched_train_data) // bptt
    progress_bar = tqdm(enumerate(range(0, batched_train_data.size(0) - 1, bptt)), total=num_batches, desc=f'Epoch {epoch}', ncols=80)
    for batch_idx, i in progress_bar:
        ### batch_idx -> (1, 2, 3, 4, ...)
        ### i -> (0, bptt, 2*bptt, ....)
        data, targets = get_batch(batched_train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()

        ## calculate the postfix description for the progress bar
        cur_loss = total_loss / (batch_idx + 1)
        ppl = math.exp(cur_loss)
        
        progress_bar.set_postfix({"loss": cur_loss, "ppl" : ppl}, refresh=True)
        
        writer.add_scalar('loss/train loss', cur_loss, global_step)
        writer.flush()
        writer.add_scalar('ppl/train perplexity', ppl, global_step)
        writer.flush()
        global_step += 1

In [28]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(eval_data) // bptt
    with torch.no_grad():
        progress_bar = tqdm(enumerate(range(0, eval_data.size(0) - 1, bptt)), total=num_batches, desc=f'Validation {epoch}', ncols=80)
        for batch_idx, i in progress_bar:
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_softmax = softmax(output)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    
    eval_loss = total_loss / (len(eval_data) - 1)
    eval_ppl = math.exp(eval_loss)

    writer.add_scalar('loss/val loss', eval_loss, global_step)
    writer.flush()
    writer.add_scalar('ppl/val perplexity', eval_ppl, global_step)
    writer.flush()

    return eval_loss

In [29]:
softmax = nn.Softmax(dim=2)
#softmax = nn.LogSoftmax(dim=2)

# Training Data

In [30]:
best_model_path = 'models/best_model_mp_new.pt'

In [31]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
best_val_loss = float('inf')
initial_epoch = 0
epochs = app_config["epochs"]
global_step = 0
best_model = None

# preload the model if exists to train more epochs

if os.path.exists(best_model_path):
    print(f"Preloading model {best_model_path}")
    state = torch.load(best_model_path)
    
    initial_epoch = state['epoch'] + 1
    model.load_state_dict(state['model_state_dict'])
    optimizer.load_state_dict(state['optimizer_state_dict'])
    global_step = state['global_step']
    best_val_loss = state['best_val_loss']
    
    print(initial_epoch, global_step, best_val_loss)

# initializing the tensorbaord log writer
writer = SummaryWriter(app_config["logs"])


for epoch in range(initial_epoch, epochs):
    train(model)
    eval_loss = evaluate(model, batched_test_data)

    # save the model if validation loss decreases

    if eval_loss < best_val_loss:
        print(f"eval perplexity : {math.exp(eval_loss)}")
        print("saving the model")
        best_val_loss = eval_loss
        best_model = copy.deepcopy(model)

        directory_path = 'models'
        # Create the directory if it doesn't exist
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
        torch.save({
                'epoch': epoch,
                'model_state_dict': best_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'global_step': global_step, 
                'best_val_loss' : best_val_loss,
            }, os.path.join(directory_path, 'best_model_mp_new.pt'))

Epoch 0: 10752it [47:01,  3.81it/s, loss=3.6, ppl=36.6]                         
Validation 0: 2556it [04:11, 10.18it/s]                                         


eval perplexity : 24.895583496259558
saving the model


Epoch 1: 10752it [46:20,  3.87it/s, loss=3.12, ppl=22.8]                        
Validation 1: 2556it [04:03, 10.50it/s]                                         


eval perplexity : 19.38394365305096
saving the model


Epoch 2: 10752it [45:18,  3.95it/s, loss=2.95, ppl=19.2]                        
Validation 2: 2556it [04:02, 10.53it/s]                                         


...

eval perplexity : 13.947761322996438
saving the model


Epoch 8: 10752it [45:06,  3.97it/s, loss=2.66, ppl=14.3]                        
Validation 8: 2556it [03:59, 10.66it/s]                                         


eval perplexity : 13.707615119019952
saving the model


Epoch 9:  69%|█████▌  | 7444/10751 [31:07<13:49,  3.99it/s, loss=2.65, ppl=14.1]


KeyboardInterrupt: 

In [26]:
lnsoftmax = nn.LogSoftmax(dim=2)

# Data Generation

In [32]:
def loadModel(best_model_path):
    global model
    if os.path.exists(best_model_path):
        print(f"Preloading model {best_model_path}")
        if torch.cuda.is_available():
            state = torch.load(best_model_path)
        else:
            state = torch.load(best_model_path, map_location=torch.device('cpu'))
        model.load_state_dict(state['model_state_dict'])
        return model
    else:
        raise Exception("Model Not Found")
        
loaded_model = loadModel(best_model_path)

Preloading model models/best_model_mp_new.pt


In [33]:
# loaded_model = model
def generator(model: nn.Module, gen_data: Tensor, no_words = 10):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        else:
            src_mask_ = src_mask[:,:]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        pred_text.append([vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        if(batch_size < 16):
            gen_data = torch.cat((gen_data[:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
            
    return pred_text





def nonnaive_generator(model: nn.Module, gen_data: Tensor, no_words = 5,k=50):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):

        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.topk(output_softmax_permuted,k ,dim=2).indices.squeeze(0)
        
        values = torch.topk(softmax(output_softmax_permuted),k ,dim=2).values
        values = values/torch.sum(values,dim = 2,keepdims = True)

        
        ind_sampled = torch.distributions.Categorical(values.squeeze(0)).sample()
        next_index = indices[-1][ind_sampled[-1]]
        

        pred_text.append([vocab.lookup_token(next_index)][0])
        if(batch_size < 15):
            gen_data = torch.cat((gen_data[:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],next_index.unsqueeze(0).unsqueeze(0)),0)
            batch_size= gen_data.size(0)
            
    return pred_text

In [34]:
import morfessor
import math

with open('models/morfessor_model.p','rb') as f:
    models = pickle.load(f)

In [35]:
def convert_to_morph(l):
    a = []
    for v in l:
        t1 = '-'.join(models[0].viterbi_segment(v)[0])
        tr = re.sub(r'[ ]+', r' ', t1)
        tr = re.sub(r'- -', r'*', tr)
        tr = re.sub(r'-[ ]+', r'*', tr)
        tr = re.sub(r'[ ]+-', r'*',tr)
        tr = re.sub(r' ', r' * ',tr)
        tr = re.sub(r'\*', r' * ',tr)
        tr = re.sub(r'  ', r' ', tr)
        tr = re.sub(r'-', r' ',tr)
        a.append(tr)
    return a

def revert_sentence(text):
#     tr = re.sub(r' ', r'',text)
    tr = re.sub(r'\*', r' ',text)
    return tr

In [36]:
# st = ['लामो समयसम्म प्रयोग गर्न सकिन्छ ।']
# st = ['तपाईंलाई कस्तो पुस्तकहरू मन']
st = ['नेपालमा आधुनिक']
st_ = convert_to_morph(st)
st_i = data_process(st_)
st_i = st_i.unsqueeze(1).to(device)

In [37]:
st_

['नेपालमा * आधुनिक']

In [38]:
z = nonnaive_generator(loaded_model, st_i,no_words = 100)

In [39]:
st[0]+ revert_sentence(''.join(z))

'नेपालमा आधुनिक प्रविधिको प्रयोग गर्ने गरेका छन् । सन् २०१७ देखि हालसम्म ३ खर्ब ८ अर्ब ९ करोड रुपैयाँ मात्र राजश्व असुली भइरहेको र त्यो भन्दा बढी राजश्व संकलन भइरहेको छ । तर ? यो सबै कुरा अहिले <unk> जस्तो लागेको छ । हाम्रो <unk> धेरै कुरा बुझेको छैन तर त्यो थाहा'

In [40]:
z

['*',
 'प्रविधिको',
 '*',
 'प्रयोग',
 '*',
 'गर्ने',
 '*',
 'गरेका',
 '*',
 'छन्',
 '*',
 '।',
 '*',
 'सन्',
 '*',
 '२०१७',
 '*',
 'देखि',
 '*',
 'हालसम्म',
 '*',
 '३',
 '*',
 'खर्ब',
 '*',
 '८',
 '*',
 'अर्ब',
 '*',
 '९',
 '*',
 'करोड',
 '*',
 'रुपैयाँ',
 '*',
 'मात्र',
 '*',
 'राजश्व',
 '*',
 'असुली',
 '*',
 'भइरहेको',
 '*',
 'र',
 '*',
 'त्यो',
 '*',
 'भन्दा',
 '*',
 'बढी',
 '*',
 'राजश्व',
 '*',
 'संकलन',
 '*',
 'भइरहेको',
 '*',
 'छ',
 '*',
 '।',
 '*',
 'तर',
 '*',
 '?',
 '*',
 'यो',
 '*',
 'सबै',
 '*',
 'कुरा',
 '*',
 'अहिले',
 '*',
 '<unk>',
 '*',
 'जस्तो',
 '*',
 'लागेको',
 '*',
 'छ',
 '*',
 '।',
 '*',
 'हाम्रो',
 '*',
 '<unk>',
 '*',
 'धेरै',
 '*',
 'कुरा',
 '*',
 'बुझेको',
 '*',
 'छैन',
 '*',
 'तर',
 '*',
 'त्यो',
 '*',
 'थाहा']